<a href="https://colab.research.google.com/github/HongJaeKwon/machine-learning/blob/master/HJK_GRID_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 패키지 싸그리 다 임포트 시킵니다.(필요없는 것도 많지만, 그냥 쓰던거 계속 붙여넣어서..;;))
import os
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from collections import deque
import gym
from google.colab import output
import time

In [2]:
env = gym.make("FrozenLake-v0")

In [7]:
action_num=env.action_space.n
state_num=env.observation_space.n
print(action_num,state_num)

4 16


In [8]:
q_table=np.zeros((state_num,action_num))

In [11]:
q_table.shape

(16, 4)

In [89]:
# 랜덤 플레이
episode_count=30000
scores = []
reward_discount_rate=0.9

for episode in range(episode_count):
    total_reward = 0
    state = env.reset()
    done=False
    while not done:
        action=env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
        total_reward=total_reward + reward
    scores.append(total_reward)
    if((episode+1)%100==0):
        print(episode+1,np.mean(scores[:-100]))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


100 nan
200 0.01
300 0.01
400 0.016666666666666666
500 0.0125
600 0.012
700 0.011666666666666667
800 0.011428571428571429
900 0.0125
1000 0.013333333333333334
1100 0.014
1200 0.013636363636363636
1300 0.014166666666666666
1400 0.014615384615384615
1500 0.014285714285714285
1600 0.016
1700 0.015625
1800 0.01588235294117647
1900 0.015
2000 0.015789473684210527
2100 0.017
2200 0.016666666666666666
2300 0.015909090909090907
2400 0.015217391304347827
2500 0.015
2600 0.0148
2700 0.015
2800 0.015555555555555555
2900 0.015357142857142857
3000 0.015172413793103448
3100 0.014666666666666666
3200 0.014516129032258065
3300 0.014375
3400 0.014242424242424242
3500 0.015
3600 0.015428571428571429
3700 0.015833333333333335
3800 0.015675675675675675
3900 0.015526315789473685
4000 0.01641025641025641
4100 0.0165
4200 0.016097560975609757
4300 0.01619047619047619
4400 0.01627906976744186
4500 0.015909090909090907
4600 0.01577777777777778
4700 0.015869565217391305
4800 0.015957446808510637
4900 0.01625
50

In [90]:
# TD UPDATE
episode_count=30000
epsilon= 0.1
q_table=np.zeros((state_num,action_num))
scores = []
reward_discount_rate=0.9
lr=0.1

for episode in range(episode_count):
    total_reward = 0
    state = env.reset()
    done=False
    while not done:
        if(np.random.rand()) < epsilon:
            action=env.action_space.sample()
        else:
            action=np.random.choice(np.where(q_table[state] == q_table[state].max())[0])
        next_state, reward, done, _ = env.step(action)
        tde=reward+reward_discount_rate*np.max(q_table[next_state])
        q_table[state][action]=q_table[state][action]+lr*(tde-q_table[state][action])

        total_reward=total_reward + reward
        state=next_state
    scores.append(total_reward)
    if((episode+1)%100==0):
        print(episode+1,np.mean(scores[:-100]))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


100 nan
200 0.0
300 0.005
400 0.01
500 0.02
600 0.024
700 0.028333333333333332
800 0.05857142857142857
900 0.07625
1000 0.10555555555555556
1100 0.117
1200 0.13727272727272727
1300 0.13833333333333334
1400 0.1476923076923077
1500 0.16
1600 0.16733333333333333
1700 0.175
1800 0.17823529411764705
1900 0.1827777777777778
2000 0.19421052631578947
2100 0.202
2200 0.20666666666666667
2300 0.21
2400 0.2091304347826087
2500 0.20833333333333334
2600 0.2108
2700 0.21576923076923077
2800 0.2174074074074074
2900 0.21785714285714286
3000 0.21862068965517242
3100 0.22166666666666668
3200 0.2235483870967742
3300 0.224375
3400 0.22636363636363635
3500 0.22705882352941176
3600 0.23
3700 0.23277777777777778
3800 0.23324324324324325
3900 0.23526315789473684
4000 0.23615384615384616
4100 0.2385
4200 0.23804878048780487
4300 0.23785714285714285
4400 0.23906976744186045
4500 0.2381818181818182
4600 0.2388888888888889
4700 0.24195652173913043
4800 0.24468085106382978
4900 0.24416666666666667
5000 0.244693877

In [95]:
# MC UPDATE
episode_count=30000
epsilon= 0.1
q_table=np.zeros((state_num,action_num))
scores = []
reward_discount_rate=0.9
lr=0.25


for episode in range(episode_count):
    total_reward = 0
    memory=[]
    state = env.reset()
    done=False
    while not done:
        if(np.random.rand()) < epsilon:
            action=env.action_space.sample()
        else:
            action=np.random.choice(np.where(q_table[state] == q_table[state].max())[0])
        next_state, reward, done, _ = env.step(action)
        i=(state,action,reward,next_state,done)
        memory.append(i)
        state=next_state
        total_reward=total_reward + reward
    G=0
    for s,a,r,n_s,d in memory[::-1]:
        G=r+reward_discount_rate * G
        # tde=r+reward_discount_rate*np.max(q_table[n_s])
        q_table[s][a]=q_table[s][a]+lr*(G-q_table[s][a])
  
    scores.append(total_reward)
    if((episode+1)%100==0):
        print(episode+1,np.mean(scores[:-100]))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


100 nan
200 0.04
300 0.05
400 0.04666666666666667
500 0.045
600 0.038
700 0.03333333333333333
800 0.03571428571428571
900 0.03375
1000 0.03333333333333333
1100 0.036
1200 0.03363636363636364
1300 0.03166666666666667
1400 0.031538461538461536
1500 0.03142857142857143
1600 0.03133333333333333
1700 0.03
1800 0.03058823529411765
1900 0.03166666666666667
2000 0.032105263157894734
2100 0.033
2200 0.03333333333333333
2300 0.03272727272727273
2400 0.033478260869565214
2500 0.03375
2600 0.0328
2700 0.033846153846153845
2800 0.034074074074074076
2900 0.03464285714285714
3000 0.037241379310344824
3100 0.037333333333333336
3200 0.037096774193548385
3300 0.0371875
3400 0.03666666666666667
3500 0.03705882352941176
3600 0.036
3700 0.035
3800 0.03513513513513514
3900 0.034210526315789476
4000 0.034102564102564105
4100 0.034
4200 0.03463414634146342
4300 0.035476190476190474
4400 0.035348837209302326
4500 0.035
4600 0.035777777777777776
4700 0.036304347826086956
4800 0.036382978723404256
4900 0.0364583